In [ ]:
!nvidia-smi

Mon Jul  4 12:03:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    32W /  70W |   4886MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-04 12:03:56--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-04 12:03:56 (34.4 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, walk_through_dir , unzip_data

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-07-04 12:03:56--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 142.250.99.128, 142.250.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   137MB/s    in 1.2s    

2022-07-04 12:03:58 (137 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [ ]:
unzip_data("10_food_classes_10_percent.zip")

In [ ]:
walk_through_dir('10_food_classes_10_percent')

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/tra

In [ ]:
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"


In [ ]:
import tensorflow as tf
IMG_SIZE = (224,224)
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir,
                                                                    image_size = IMG_SIZE,
                                                                    label_mode = "categorical")
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir,
                                                                    image_size = IMG_SIZE,
                                                                    label_mode = "categorical")

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [ ]:
train_data_10_percent.class_names


['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [ ]:
for images, labels in train_data_10_percent.take(1):
  print(images, labels)


In [ ]:
efficient_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
)
efficient_model.trainable = False

In [ ]:
len(train_data_10_percent)

24

In [ ]:
inputs = tf.keras.Input((224,224,3))

x = efficient_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(10)(x)
print(output.shape)

model_y = tf.keras.Model(inputs, output)

(None, 10)


In [ ]:
model_y.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                        optimizer = tf.keras.optimizers.Adam(),
                        metrics = ["accuracy"])

In [ ]:
model_y.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 10)                12810     
                                                                 
Total params: 4,062,381
Trainable params: 12,810
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
model_y.fit(train_data_10_percent,
            epochs = 20,
            steps_per_epoch = len(train_data_10_percent),
            validation_data = test_data,
            validation_steps = len(test_data))

In [ ]:
len(train_data_10_percent)

24

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False
inputs = tf.keras.layers.Input(shape = (224,224,3),name = "input_layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(10, activation = 'relu')(x)
model_0 = tf.keras.Model(inputs, outputs)
model_0.compile(loss = 'categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])
h = model_0.fit(train_data_10_percent,
                epochs = 10,
            steps_per_epoch = len(train_data_10_percent),
            validation_data = test_data,
            validation_steps = len(test_data))


Epoch 1/10
24/24 [==============================] - 15s 378ms/step - loss: 4.1075 - accuracy: 0.3520 - val_loss: 1.8385 - val_accuracy: 0.5824
Epoch 2/10
24/24 [==============================] - 8s 323ms/step - loss: nan - accuracy: 0.6627 - val_loss: nan - val_accuracy: 0.6944
Epoch 3/10
24/24 [==============================] - 8s 323ms/step - loss: nan - accuracy: 0.7400 - val_loss: nan - val_accuracy: 0.7484
Epoch 4/10
24/24 [==============================] - 8s 323ms/step - loss: nan - accuracy: 0.7720 - val_loss: nan - val_accuracy: 0.7484
Epoch 5/10
24/24 [==============================] - 8s 323ms/step - loss: nan - accuracy: 0.8000 - val_loss: nan - val_accuracy: 0.7460
Epoch 6/10
24/24 [==============================] - 8s 322ms/step - loss: nan - accuracy: 0.7667 - val_loss: nan - val_accuracy: 0.7304
Epoch 7/10
24/24 [==============================] - 8s 322ms/step - loss: nan - accuracy: 0.8120 - val_loss: nan - val_accuracy: 0.7712
Epoch 8/10
24/24 [=======================

In [ ]:
!!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip

In [ ]:
train_dir = "10_food_classes_1_percent/train"
test_dir = "10_food_classes_1_percent/test"
unzip_data('10_food_classes_1_percent.zip')

In [ ]:
walk_through_dir("10_food_classes_1_percent")

There are 2 directories and 0 images in '10_food_classes_1_percent'.
There are 10 directories and 0 images in '10_food_classes_1_percent/train'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/ramen'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/chicken_curry'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/pizza'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/hamburger'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/grilled_salmon'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/chicken_wings'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/steak'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/fried_rice'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/sushi'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/ice_cream'.
There a

AttributeError: ignored

In [ ]:


input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False

# Creat input layer
inputs = tf.keras.layers.Input(shape = input_shape)

x = data_augmentation(inputs)
x = base_model(x, training = False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers. Dense(10,activation= 'relu')


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
from tensorflow.keras.models import Sequential


In [ ]:
data_augmentation = Sequential([
   preprocessing.RandomFlip("horizontal"),
   preprocessing.RandomHeight(0.2),
   preprocessing.RandomWidth(0.2),
   preprocessing.RandomZoom(0.2),
   preprocessing.RandomRotation(0.2),
], name = "data_augmentation")

In [ ]:
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False 

inputs = layers.Input(shape = input_shape, name = "input_layer")
x = data_augmentation(inputs)
x = base_model(x , training = False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10 , activation = 'relu')(x)
model_2 = tf.keras.Model(inputs, outputs)
model_2.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
check_point = "ten_percent_model_checkpoint_weights/checkpoint.ckpt"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = check_point,
                                                        save_weights_only = True,
                                                        save_best_only = False,
                                                        save_freq = "epoch",
                                                        verbose = 1)

In [ ]:
initial_epoch = 5

In [ ]:
h = model_2.fit(train_data_10_percent,
            epochs = initial_epoch,
            steps_per_epoch = len(train_data_10_percent),
            validation_data = test_data,
            validation_steps = len(test_data),
            callbacks = [checkpoint_callback])

Epoch 1/5
24/24 [==============================] - ETA: 0s - loss: 4.3107 - accuracy: 0.2067
Epoch 1: saving model to ten_percent_model_checkpoint_weights/checkpoint.ckpt
24/24 [==============================] - 17s 477ms/step - loss: 4.3107 - accuracy: 0.2067 - val_loss: 2.4594 - val_accuracy: 0.4076
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 2.2477 - accuracy: 0.3893
Epoch 2: saving model to ten_percent_model_checkpoint_weights/checkpoint.ckpt
24/24 [==============================] - 10s 427ms/step - loss: 2.2477 - accuracy: 0.3893 - val_loss: 1.7861 - val_accuracy: 0.5148
Epoch 3/5
24/24 [==============================] - ETA: 0s - loss: 1.9027 - accuracy: 0.4880
Epoch 3: saving model to ten_percent_model_checkpoint_weights/checkpoint.ckpt
24/24 [==============================] - 10s 406ms/step - loss: 1.9027 - accuracy: 0.4880 - val_loss: 1.6070 - val_accuracy: 0.5820
Epoch 4/5
24/24 [==============================] - ETA: 0s - loss: 1.6647 - accuracy: 0.546

In [ ]:
base_model.summary()

In [ ]:
model_2.layers

In [ ]:
for l in model_2.layers:
  print(l , l.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fb6811cda90> True
<keras.engine.sequential.Sequential object at 0x7fb6811f6410> True
<keras.engine.functional.Functional object at 0x7fb680d2a590> False
<keras.layers.pooling.GlobalAveragePooling2D object at 0x7fb6811cdc10> True
<keras.layers.core.dense.Dense object at 0x7fb680ce9110> True


In [ ]:
base_model.trainable = True
for l in base_model.layers[:-10]:
  l.trainable = False

model_2.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
                loss = "categorical_crossentropy",
                metrics = ["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
fine_tune = initial_epoch + 5
history_fine_tune = model_2.fit(train_data_10_percent,
                                epochs = fine_tune,
                                steps_per_epoch = len(train_data_10_percent),
                                validation_data = test_data,
                                validation_steps = len(test_data),
                                initial_epoch = h.epoch[-1])

Epoch 5/10
24/24 [==============================] - 18s 476ms/step - loss: 1.6831 - accuracy: 0.6173 - val_loss: nan - val_accuracy: 0.7016
Epoch 6/10
24/24 [==============================] - 10s 410ms/step - loss: 1.5524 - accuracy: 0.6920 - val_loss: nan - val_accuracy: 0.7212
Epoch 7/10
24/24 [==============================] - 10s 415ms/step - loss: 1.3493 - accuracy: 0.7027 - val_loss: nan - val_accuracy: 0.7400
Epoch 8/10
24/24 [==============================] - 11s 441ms/step - loss: 1.2751 - accuracy: 0.7320 - val_loss: nan - val_accuracy: 0.7352
Epoch 9/10
24/24 [==============================] - 10s 396ms/step - loss: 1.3224 - accuracy: 0.7173 - val_loss: nan - val_accuracy: 0.7072
Epoch 10/10
24/24 [==============================] - 10s 408ms/step - loss: 1.2814 - accuracy: 0.7573 - val_loss: nan - val_accuracy: 0.7676


fine tuning on all the data

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

--2022-07-04 12:12:40--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.20.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip.1’

10_food_classes_all 100%[===================>] 495.13M  63.8MB/s    in 7.8s    

2022-07-04 12:12:47 (63.5 MB/s) - ‘10_food_classes_all_data.zip.1’ saved [519183241/519183241]



In [ ]:
unzip_data('10_food_classes_all_data.zip')

In [ ]:
train_dir = '10_food_classes_all_data/train'
train_dir = '10_food_classes_all_data/test'


In [ ]:
walk_through_dir('10_food_classes_all_data')

There are 2 directories and 0 images in '10_food_classes_all_data'.
There are 10 directories and 0 images in '10_food_classes_all_data/train'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/ramen'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/chicken_curry'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/pizza'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/hamburger'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/grilled_salmon'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/chicken_wings'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/steak'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/fried_rice'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/sushi'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/ice_cream'.

In [ ]:
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all =tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                    label_mode = "categorical",
                                                                    image_size = IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode = "categorical",
                                                                image_size = IMG_SIZE)

Found 2500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [ ]:
model_2.load_weights(check_point)

In [ ]:
model_2.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
               optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
               metrics = ["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
fine_tune = initial_epoch + 5 
history_fine_all_data = model_2.fit(train_data_all,
                                    epochs = fine_tune,
                                    steps_per_epoch = len(train_data_all),
                                    validation_data  = test_data,
                                    validation_steps = len(test_data),
                                    initial_epoch = h.epoch[-1],
                                    callbacks = create_tensorboard_callback(dir_name = "transfer_learning",
                                                                           experiment_name = "full_train_data"),
                                    )

Saving TensorBoard log files to: transfer_learning/full_train_data/20220704-121256
Epoch 5/10
79/79 [==============================] - 26s 254ms/step - loss: nan - accuracy: 0.6516 - val_loss: 1.1058 - val_accuracy: 0.7528
Epoch 6/10
79/79 [==============================] - 18s 226ms/step - loss: 1.3637 - accuracy: 0.6848 - val_loss: 1.0011 - val_accuracy: 0.7556
Epoch 7/10
79/79 [==============================] - 18s 225ms/step - loss: 1.1888 - accuracy: 0.7384 - val_loss: 0.9487 - val_accuracy: 0.7864
Epoch 8/10
79/79 [==============================] - 18s 230ms/step - loss: 1.0952 - accuracy: 0.7536 - val_loss: nan - val_accuracy: 0.8060
Epoch 9/10
79/79 [==============================] - 18s 222ms/step - loss: nan - accuracy: 0.7540 - val_loss: 0.8884 - val_accuracy: 0.8092
Epoch 10/10
79/79 [==============================] - 17s 216ms/step - loss: nan - accuracy: 0.7860 - val_loss: nan - val_accuracy: 0.8372
